<a href="https://colab.research.google.com/github/21Nuri/AI_Project_team/blob/main/ai_%ED%94%84%EC%A0%9D_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import

import os
from datetime import datetime, timedelta
import re
import shutil
from PIL import Image, ImageEnhance
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.image import pad_to_bounding_box, central_crop, resize
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import to_categorical

In [ ]:
# 폴더 이름 바꾸기

parent_path = "/content/drive/MyDrive/Colab Notebooks/팀프로젝트/뿌리 데이터(저해상도)"  # 상위 폴더 경로 수정하기
folder_names = ['test', 'train']

for folder_name in folder_names:
    file_path = os.path.join(parent_path, folder_name)
    file_names = os.listdir(file_path)

    n = 1
    for i in file_names:
        src = os.path.join(file_path, i)

        # 변경된 파일명 생성
        new_file_name = f"{n}"

        dst = os.path.join(file_path, new_file_name)
        os.rename(src, dst)
        n += 1

In [ ]:
# 파일 이름 변경하기

parent_directory = "/content/drive/MyDrive/Colab Notebooks/팀프로젝트/뿌리 데이터(저해상도)"  # 상위 폴더 경로 수정하기
folder_names = ['test', 'train']

for folder_name in folder_names:
    folder_path = os.path.join(parent_directory, folder_name)

    # test/train 폴더 내의 파일과 폴더 순회
    for directory in os.listdir(folder_path):
        # 폴더의 절대 경로
        sub_folder_path = os.path.join(folder_path, directory)

        # readme.txt 파일의 경로
        readme_file_path = os.path.join(sub_folder_path, "readme.txt")

        # readme.txt 파일이 있는지 확인
        if os.path.isfile(readme_file_path):
            # 폴더 삭제
            shutil.rmtree(sub_folder_path)
            print(f"폴더 삭제: {sub_folder_path}")

    for directory in os.listdir(folder_path):
        # 폴더의 절대 경로
        sub_folder_path = os.path.join(folder_path, directory)

        # 디렉토리 내의 파일 목록 가져오기
        file_list = os.listdir(sub_folder_path)

        # 파일 목록을 시간 순으로 정렬하기 전에 파일이름이 규정과 다른 경우 해당 파일 삭제
        for file_name in file_list:
            # 파일의 절대 경로
            file_path = os.path.join(sub_folder_path, file_name)

            # 파일명이 규정과 다른 경우 파일 삭제
            file_name_split = file_name.split("_")[1]
            if not re.match(r'\d{12}\.jpg', file_name_split):
                os.remove(file_path)
                print(f"삭제된 파일: {file_name}")

        # 디렉토리 내의 파일 목록 가져오기
        file_list = os.listdir(sub_folder_path)

        # 파일 목록을 시간 순으로 정렬
        sorted_files = sorted(file_list, key=lambda x: int(x.split("_")[1].split(".")[0]))

        # 가장 이른 시간의 파일명 추출
        first_file = sorted_files[0]

        # 가장 이른 시간 추출
        first_time = datetime.strptime(first_file.split("_")[1].split(".")[0], "%y%m%d%H%M%S")

        # 파일명 변경
        for file in sorted_files:
            # 파일의 현재 경로
            file_path = os.path.join(sub_folder_path, file)

            # 파일의 확장자
            file_extension = os.path.splitext(file)[1]

            # 파일의 시간 추출
            file_time = datetime.strptime(file.split("_")[1].split(".")[0], "%y%m%d%H%M%S")

            # 시간 차이 계산 (분 단위)
            time_difference = int((file_time - first_time).total_seconds() // 60)

            # 파일이 .jpg가 아닌 경우 폴더 건너뛰기
            if file_extension.lower() != ".jpg":
                continue

            # 시간 간격에 따라 분류
            if 0 <= time_difference < 720:
                time_category = 0
            elif 720 <= time_difference < 1440:
                time_category = 1
            elif 1440 <= time_difference < 2160:
                time_category = 2
            elif 2160 <= time_difference < 2880:
                time_category = 3
            elif 2880 <= time_difference < 3660:
                time_category = 4
            elif 3660 <= time_difference < 4320:
                time_category = 5
            else:
                time_category = 6

            # 변경된 파일명 생성
            new_file_name = f"{directory}_{time_difference}_{time_category}{file_extension}"

            # 새로운 파일명으로 파일 이동 또는 복사
            new_file_path = os.path.join(sub_folder_path, new_file_name)
            os.rename(file_path, new_file_path)

            print(f"파일명 변경: {file} -> {new_file_name}")

In [ ]:
# 폴더에 있는 사진들 하나로 모으기

parent_directory = "//content/drive/MyDrive/Colab Notebooks/팀프로젝트/뿌리 데이터(저해상도)"  # 상위 폴더 경로 수정하기
folder_names = ['test', 'train']

for folder_name in folder_names:
    parent_folder_path = os.path.join(parent_directory, folder_name)

    # 상위 폴더 내의 모든 하위 폴더에 대해 반복
    for folder_name in os.listdir(parent_folder_path):
        folder_path = os.path.join(parent_folder_path, folder_name)
        
        # 폴더인 경우에만 처리
        if os.path.isdir(folder_path):
            # 하위 폴더 내의 파일들을 상위 폴더로 이동
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                shutil.move(file_path, parent_folder_path)
            
            # 하위 폴더 삭제
            shutil.rmtree(folder_path)

In [ ]:
# 전체 사진 크기 동일한지 확인하기

# 이미지가 저장된 폴더 경로
parent_directory = "/content/drive/MyDrive/Colab Notebooks/팀프로젝트/뿌리 데이터(저해상도)"  # 상위 폴더 경로 수정하기
folder_names = ['test', 'train']

for folder_name in folder_names:
    folder_path = os.path.join(parent_directory, folder_name)

    # 디렉토리 내의 파일 목록 가져오기
    file_list = os.listdir(folder_path)

    # 첫 번째 이미지의 크기
    first_image_path = os.path.join(folder_path, file_list[0])
    first_image = Image.open(first_image_path)
    first_width, first_height = first_image.size

    # 모든 이미지의 크기 확인
    for file_name in file_list[1:]:
        file_path = os.path.join(folder_path, file_name)
        image = Image.open(file_path)
        width, height = image.size
        
        if (width, height) != (first_width, first_height):
            print(f"이미지 크기 불일치: {file_name}")
            break
    else:
        print(f"{folder_name} 모든 이미지 크기 일치")

In [ ]:
# 대비 조절

# 폴더 경로
parent_directory = "/content/drive/MyDrive/Colab Notebooks/팀프로젝트/뿌리 데이터(저해상도)"  # 상위 폴더 경로 수정하기
folder_names = ['test', 'train']

for folder_name in folder_names:
    folder_path = os.path.join(parent_directory, folder_name)

    # 디렉토리 내의 파일 목록 가져오기
    file_list = os.listdir(folder_path)

    def adjust_contrast(image, factor):
        enhancer = ImageEnhance.Contrast(image)
        adjusted_image = enhancer.enhance(factor)
        return adjusted_image

    # 이미지 대비 조절
    for file in file_list:
        # 파일의 현재 경로
        file_path = os.path.join(folder_path, file)

        # 이미지 열기
        image = Image.open(file_path)

        # 대비 조정
        adjusted_image = adjust_contrast(image, 1.5)  # 1.5는 대비 조정 계수입니다.

        # 조절된 이미지 저장
        adjusted_image.save(file_path)

In [ ]:
#패딩 추가

from tensorflow.keras.preprocessing import image

# 폴더 경로
parent_directory = "/content/drive/MyDrive/Colab Notebooks/팀프로젝트/뿌리 데이터(저해상도)"  # 상위 폴더 경로 수정하기
folder_names = ['test', 'train']

for folder_name in folder_names:
    folder_path = os.path.join(parent_directory, folder_name)

    # 디렉토리 내의 파일 목록 가져오기
    file_list = os.listdir(folder_path)

    for file in file_list:
    # 이미지 파일 경로 생성
        file_path = os.path.join(folder_path, file)

        #이미지 불러오기
        bgd = image.load_img(file_path)
        bgd_vector = np.asarray(image.img_to_array(bgd))
        bgd_vector = bgd_vector/255
        
        # #이미지 형태 확인 
        # bgd_vector.shape

        #이미지의 변경할 크기 설정 
        target_height = 320
        target_width = 320
        
        #현재 이미지의 크기 지정 
        source_height = bgd_vector.shape[0]
        source_width = bgd_vector.shape[1]
        
        #padding 실시 : pad_to_bounding_box 사용 
        bgd_vector_pad = pad_to_bounding_box(bgd_vector, 
                                            int((target_height-source_height)/2), 
                                            int((target_width-source_width)/2), 
                                            target_height, 
                                            target_width)

        image.save_img(file_path, bgd_vector_pad)

In [ ]:
# 사진 crop하기

from tensorflow.keras.preprocessing import image

# 폴더 경로
parent_directory = "/content/drive/MyDrive/Colab Notebooks/팀프로젝트/뿌리 데이터(저해상도)"  # 상위 폴더 경로 수정하기
folder_names = ['test', 'train']

for folder_name in folder_names:
    folder_path = os.path.join(parent_directory, folder_name)

    # 디렉토리 내의 파일 목록 가져오기
    file_list = os.listdir(folder_path)

    for file in file_list:
        # 이미지 파일 경로 생성
        file_path = os.path.join(folder_path, file)

        image = Image.open(file_path)

        img_cropped = image.crop((73,73,247,247))

        # 조절된 이미지 저장
        img_cropped.save(file_path)

        # plt.imshow(img_cropped)
        # plt.show()

In [ ]:
# 사진 확인하기

image = Image.open("/content/drive/MyDrive/Colab Notebooks/팀프로젝트/뿌리 데이터(저해상도)/test/2_0_0.jpg")

plt.imshow(image)
plt.show()

In [ ]:
# x_train / test 배열로 변환후 정규화

# 이미지가 있는 폴더 경로
parent_directory = "/content/drive/MyDrive/Colab Notebooks/팀프로젝트/뿌리 데이터(저해상도)"  # 상위 폴더 경로 수정하기
folder_names = ['test', 'train']

for folder_name in folder_names:
    folder_path = os.path.join(parent_directory, folder_name)

    # 이미지 확장자 (예: .jpg, .png 등)
    image_extension = ".jpg"

    # 이미지 크기 설정
    image_size = (140, 140)

    # 이미지 파일 목록 가져오기
    file_list = os.listdir(folder_path)

    # 이미지 데이터와 레이블 초기화
    images = []
    labels = []

    # 이미지 파일 순회
    for file_name in file_list:
        # 이미지 파일 경로
        image_path = os.path.join(folder_path, file_name)

        # 이미지 파일 로드
        image = cv2.imread(image_path)

        # 이미지 크기 변경
        image = cv2.resize(image, image_size)

        # 이미지 데이터 추가
        images.append(image)

        # 이미지 파일명에서 레이블 추출
        label = int(file_name.split("_")[2].split(".")[0])  # 예시: 파일명이 "1_20_0.jpg"라면 0이 레이블로 설정됨

        # 레이블 추가
        labels.append(label)

    if folder_name == 'train':
        # 이미지 데이터와 레이블을 NumPy 배열로 변환
        x_train = np.array(images)
        y_train = np.array(labels)

        # 이미지 데이터 정규화
        x_train = x_train.astype('float32') / 255.

        # 이미지 시각화
        n = 2
        plt.imshow(x_train[n], cmap='Greys', interpolation='nearest')
        plt.show()
    
    elif folder_name == 'test':
        # 이미지 데이터와 레이블을 NumPy 배열로 변환
        x_test = np.array(images)
        y_test = np.array(labels)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 140, 140, 3)
x_test = x_test.reshape(x_test.shape[0], 140, 140, 3)
 
input_shape = (140, 140, 3)
input_shape

In [ ]:
x_train.shape[0]
y_train.shape[0]

In [ ]:
y_train[0:10]
num_classes = 7
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_train[0:10]

In [ ]:
np.random.seed(7)

In [ ]:
# 모델 1 (인식률이 현저하게 낮음)

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))

# 모델 출력 레이어 수정
num_classes = 7  # 클래스 수에 맞게 설정
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# 모델 2 (인식률은 모델 1에 비해 더욱 높지만, Ram을 많이 사용함)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# 모델 3 (Ram 부족으로 불가능)

from tensorflow.keras.layers import BatchNormalization

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# 모델에 데이터셋 학습시키기

batch_size=128
epochs=15

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

In [ ]:
# Test 인식률

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# 그래프 확인하기

plt.figure(figsize=(12,8))
# plt.plot(hist.history['loss'])
# plt.plot(hist.history['val_loss'])
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
# plt.legend(['loss', 'val_loss', 'acc', 'val_acc'])
plt.legend(['acc', 'val_acc'])
plt.show()